In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
!pip install -q tiktoken
from utils import load_gpt2_params_from_tf_ckpt
from utils import tldr
from utils import text_to_token_ids
from utils import token_ids_to_text

In [2]:
from gpt2u import GPT2u

In [3]:
config124M = {'n_embd': 768, 'n_vocab': 50257, 'n_ctx': 1024, 'n_layer': 12, 'n_head': 12}
gpt2u=GPT2u(config124M)

In [4]:
model_dir="openai_gpt2_weights/124M"
tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, config124M)

In [5]:
gpt2u.embedding.word_embedding = tf.Variable(params['wte'], name="word_embedding")
gpt2u.embedding.position_embedding = tf.Variable(params['wpe'], name="position_embedding")

for b in range(12):
    print(f"loading block {b}")
    # Attention layer_norm
    gpt2u.transformer.blocks[b].attention.layer_norm.beta = tf.Variable(params["blocks"][b]["ln_1"]["b"], name=f"transformer.blocks-{b}.attention.layer_norm.beta")
    gpt2u.transformer.blocks[b].attention.layer_norm.gamma = tf.Variable(params["blocks"][b]["ln_1"]["g"], name=f"transformer.blocks-{b}.attention.layer_norm.gamma")

    q_w, k_w, v_w = np.split((params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
    q_b, k_b, v_b = np.split((params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
    gpt2u.transformer.blocks[b].attention.self_attention.query_layer.w = tf.Variable(q_w, name=f"transformer.blocks-{b}.attention.self_attention.query_layer.w")
    gpt2u.transformer.blocks[b].attention.self_attention.query_layer.b = tf.Variable(q_b, name=f"transformer.blocks-{b}.attention.self_attention.query_layer.b")
    gpt2u.transformer.blocks[b].attention.self_attention.key_layer.w = tf.Variable(k_w, name=f"transformer.blocks-{b}.attention.self_attention.key_layer.w")
    gpt2u.transformer.blocks[b].attention.self_attention.key_layer.b = tf.Variable(k_b, name=f"transformer.blocks-{b}.attention.self_attention.key_layer.b")
    gpt2u.transformer.blocks[b].attention.self_attention.value_layer.w = tf.Variable(v_w, name=f"transformer.blocks-{b}.attention.self_attention.value_layer.w")
    gpt2u.transformer.blocks[b].attention.self_attention.value_layer.b = tf.Variable(v_b, name=f"transformer.blocks-{b}.attention.self_attention.value_layer.b")
    gpt2u.transformer.blocks[b].attention.projection.w = tf.Variable(params["blocks"][b]["attn"]["c_proj"]["w"], name=f"transformer.blocks-{b}.attention.projection.w")
    gpt2u.transformer.blocks[b].attention.projection.b = tf.Variable(params["blocks"][b]["attn"]["c_proj"]["b"], name=f"transformer.blocks-{b}.attention.projection.b")

    gpt2u.transformer.blocks[b].mlp.layer_norm.beta = tf.Variable(params["blocks"][b]["ln_2"]["b"], name=f"transformer.blocks-{b}.mlp.layer_norm.beta")
    gpt2u.transformer.blocks[b].mlp.layer_norm.gamma = tf.Variable(params["blocks"][b]["ln_2"]["g"], name=f"transformer.blocks-{b}.mlp.layer_norm.gamma")
    gpt2u.transformer.blocks[b].mlp.perceptron.w = tf.Variable(params["blocks"][b]["mlp"]["c_fc"]["w"], name=f"transformer.blocks-{b}.mlp.perceptron.w")
    gpt2u.transformer.blocks[b].mlp.perceptron.b = tf.Variable(params["blocks"][b]["mlp"]["c_fc"]["b"], name=f"transformer.blocks-{b}.mlp.perceptron.b")                
    gpt2u.transformer.blocks[b].mlp.projection.w = tf.Variable(params["blocks"][b]["mlp"]["c_proj"]["w"], name=f"transformer.blocks-{b}.mlp.projection.w")
    gpt2u.transformer.blocks[b].mlp.projection.b = tf.Variable(params["blocks"][b]["mlp"]["c_proj"]["b"], name=f"transformer.blocks-{b}.mlp.projection.b")     
gpt2u.transformer.layer_norm.beta = tf.Variable(params["b"], name="transformer.layer_norm.beta")
gpt2u.transformer.layer_norm.gamma = tf.Variable(params["g"], name="transformer.layer_norm.gamma")

loading block 0
loading block 1
loading block 2
loading block 3
loading block 4
loading block 5
loading block 6
loading block 7
loading block 8
loading block 9
loading block 10
loading block 11


In [6]:
x_effort = tf.constant([[6109, 3626, 6100, 345]])

In [7]:
y = gpt2u(x_effort)
tldr(y)

.. final logits: float32 (1, 4, 50257) [-35.521214 -34.924126 -38.39469 ]


'float32 (1, 4, 50257) [-35.521214 -34.924126 -38.39469 ]'

In [8]:
len(gpt2u.variables)

196

In [9]:
chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model=gpt2u)
checkpoint.write(chkp_path)

'my_checkpoint'

In [10]:
config124M = {'n_embd': 768, 'n_vocab': 50257, 'n_ctx': 1024, 'n_layer': 12, 'n_head': 12}
gpt2z=GPT2u(config124M)


In [11]:
checkpoint = tf.train.Checkpoint(model=gpt2z)

In [12]:
checkpoint.restore("my_checkpoint")

In [13]:
len(gpt2z.variables)

196

In [14]:
y = gpt2z(x_effort)
tldr(y)

.. final logits: float32 (1, 4, 50257) [-35.521214 -34.924126 -38.39469 ]


'float32 (1, 4, 50257) [-35.521214 -34.924126 -38.39469 ]'

In [15]:
#
#
#
#

In [16]:
len(gpt2z.trainable_variables)

196

In [17]:
from utils import generate_text_simple

In [18]:
token_ids = generate_text_simple(gpt2z, idx=x_effort, max_new_tokens=5, context_size=256)

.. final logits: float32 (1, 4, 50257) [-35.521214 -34.924126 -38.39469 ]
.. final logits: float32 (1, 5, 50257) [-35.521214 -34.924126 -38.39469 ]
.. final logits: float32 (1, 6, 50257) [-35.521214 -34.924126 -38.3947  ]
.. final logits: float32 (1, 7, 50257) [-35.521214 -34.924126 -38.3947  ]
.. final logits: float32 (1, 8, 50257) [-35.521263 -34.924175 -38.394753]


In [19]:
assert np.array_equal(token_ids.numpy(), np.array([[6109, 3626, 6100,  345, 2651,   13,  198,  198,  464]]))

In [20]:
token_ids

<tf.Tensor: shape=(1, 9), dtype=int64, numpy=array([[6109, 3626, 6100,  345, 2651,   13,  198,  198,  464]])>

In [23]:
start_context = "Every effort moves you"
token_ids = generate_text_simple(
    model=gpt2z,
    idx=text_to_token_ids(start_context),
    max_new_tokens=30,
    # max_new_tokens=30,
    context_size=256
)

print("Output text:\n", token_ids_to_text(token_ids))

.. final logits: float32 (1, 4, 50257) [-35.521214 -34.924126 -38.39469 ]
.. final logits: float32 (1, 5, 50257) [-35.521214 -34.924126 -38.39469 ]
.. final logits: float32 (1, 6, 50257) [-35.521214 -34.924126 -38.3947  ]
.. final logits: float32 (1, 7, 50257) [-35.521214 -34.924126 -38.3947  ]
.. final logits: float32 (1, 8, 50257) [-35.521263 -34.924175 -38.394753]
.. final logits: float32 (1, 9, 50257) [-35.521263 -34.924175 -38.394753]
.. final logits: float32 (1, 10, 50257) [-35.521236 -34.92415  -38.39473 ]
.. final logits: float32 (1, 11, 50257) [-35.521236 -34.92415  -38.39473 ]
.. final logits: float32 (1, 12, 50257) [-35.521236 -34.92415  -38.39473 ]
.. final logits: float32 (1, 13, 50257) [-35.521236 -34.92415  -38.39473 ]
.. final logits: float32 (1, 14, 50257) [-35.521236 -34.92415  -38.39473 ]
.. final logits: float32 (1, 15, 50257) [-35.521236 -34.92415  -38.39473 ]
.. final logits: float32 (1, 16, 50257) [-35.521236 -34.92415  -38.39473 ]
.. final logits: float32 (1, 17